In [1]:
import geopandas
from matplotlib import pyplot as plt
import pandas as pd
from shapely.ops import cascaded_union
from shapely import wkt
#pip install folium matplotlib mapclassify

# Setting up our environment

In [2]:
gdf_world = geopandas.read_file('Data/countries.geojson')
fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(15)

<Figure size 1080x720 with 0 Axes>

In [3]:
#Load the europe_regions list
europe_regions = pd.read_csv('Data/Europe_Regions.csv',delimiter=';')

#Left join the europe_regions list to the world map geo dataframe. 
#It is important to consider the output will be a pandas data frame. 
df = gdf_world.merge(europe_regions, left_on='ISO_A3', right_on = 'ISO3',  how='left')

#Make sure all rows that have not been matched are erased. 
#Change the name of the first column to make it easier to understand and just keep the columns of interest.
df = df.dropna(axis=0)
df = df.rename(columns={"ADMIN":"country_name"})
df = df[["country_name","ISO3","geometry","Region"]]
df.reset_index(inplace=True, drop=True)
df.head()

#The corresponding pandas dataframe is converted into a geo dataframe.
gdf = geopandas.GeoDataFrame(df, crs='epsg:4326')
gdf.head()

,country_name,ISO3,geometry,Region
0,Austria,AUT,"POLYGON ((16.94504 48.60417, 16.91052 48.63078...",Central Europe
1,Belgium,BEL,"POLYGON ((5.99491 50.74993, 5.79317 50.75215, ...",Benelux
2,Switzerland,CHE,"POLYGON ((9.54748 47.53455, 9.27321 47.65009, ...",Central Europe
3,Czech Republic,CZE,"POLYGON ((14.81039 50.85845, 14.77536 50.81297...",Central Europe
4,Germany,DEU,"MULTIPOLYGON (((13.98618 54.07123, 13.80616 54...",Central Europe


In [4]:
#gdf.explore()

# How to merge different polygons

In [5]:
gdf.explore("Region")

# Merging process

## Define a function that returns the corresponding boundary of the aggregation of polygons

In [6]:
southern_europe_df = gdf.loc[(gdf['Region'] == 'Southern Europe')]
southern_europe_df.head()

,country_name,ISO3,geometry,Region
6,Spain,ESP,"MULTIPOLYGON (((-17.88512 27.79792, -17.89647 ...",Southern Europe
8,France,FRA,"MULTIPOLYGON (((55.50131 -20.87322, 55.40797 -...",Southern Europe
12,Italy,ITA,"MULTIPOLYGON (((11.98064 36.82709, 11.93735 36...",Southern Europe
17,Portugal,PRT,"MULTIPOLYGON (((-17.06798 32.81415, -17.20027 ...",Southern Europe


In [7]:
def boundary_generator(df,string):
    polygons = gdf.loc[(gdf['Region'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons

# Southern Europe
boundary_southern_europe = boundary_generator(df,'Southern Europe')

#A geopandas is generated
data = {'Region': [
                            'Southern Europe'
                           ],
        'geometry': [
                     boundary_southern_europe
                    ],
       'code':[
               1
              ],
       }

df_southern = pd.DataFrame(data)

gp_southern = geopandas.GeoDataFrame(df_southern, crs='epsg:4326')
#gp_southern.explore()

/Users/josepferrersanchez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


## A DataFrame with all 4 European regions is created.

In [8]:
#All boundaries are computed.
boundaries = dict((str(ii), boundary_generator(df,ii)) for ii in set(df['Region'].values))

# A dictionary is generated containing the region name, the region geometry and its corresponding code
data = {'Region':   list(boundaries.keys()),
        'geometry': list(boundaries.values()),
       'code':      list(range(0,len(boundaries))),
       }
 
# Create DataFrame
df_regions = pd.DataFrame(data)

df_regions = geopandas.GeoDataFrame(df_regions, crs='epsg:4326')
df_regions.explore()

/Users/josepferrersanchez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [9]:
df_regions

,Region,geometry,code
0,Northern Europe,"MULTIPOLYGON (((3.42213 -54.38014, 3.48666 -54...",0
1,Benelux,"MULTIPOLYGON (((-68.28222 12.15990, -68.30256 ...",1
2,Central Europe,"MULTIPOLYGON (((10.36917 46.67240, 10.46663 46...",2
3,Southern Europe,"MULTIPOLYGON (((-51.94871 3.72510, -51.98842 3...",3
